In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.1/7.1 MB 80.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 29.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.2 MB/s eta 0:00:00


#Easy RWKV:

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install transformers accelerate optimum

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "ybelkada/rwkv-raven-1b5"
model = AutoModelForCausalLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

In [ ]:
question = "Tell me about ravens"
prompt = f"### Instruction: {question}\n### Response:"



In [ ]:
def rave_respond(question, prompt):
  inputs = tokenizer(prompt, return_tensors="pt").to(0)
  output = model.generate(inputs["input_ids"], max_new_tokens=100)

  return (tokenizer.decode(output[0].tolist(), skip_special_tokens=True))

In [ ]:
print(rave_respond(question,prompt))

In [ ]:
%timeit rave_respond(question,prompt)

#Easy seq2seq:

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM,GenerationConfig

In [ ]:
model_name ='google/flan-t5-base'

In [ ]:
text = 'what the fuck is happening?'

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokens = tokenizer(text, return_tensors="pt")
config = GenerationConfig(max_new_tokens=200)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
outputs = model.generate(**tokens, generation_config=config)
print(tokenizer.batch_decode(outputs, skip_special_tokens=True))

['a sex riot']


#Easy CausalLM (GPT2-like)

In [ ]:
model_checkpoint = "distilgpt2"

In [ ]:
from transformers import AutoTokenizer,AutoModelForCausalLM #,   import Trainer, TrainingArguments
    
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

In [ ]:
text = "People are the"

In [ ]:
tokens = tokenizer(text, return_tensors="pt")
print(tokens)

{'input_ids': tensor([[8061,  389,  262]]), 'attention_mask': tensor([[1, 1, 1]])}


In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_checkpoint).to('cuda')

In [ ]:
import torch

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
input_ids = tokenizer(text, return_tensors="pt")["input_ids"].to('cuda')
output = model.generate(input_ids, max_new_tokens=256, do_sample=True)
print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


People are the people who would be affected by these changes, but the more I feel about these changes, the more I feel about those changes.‪<|endoftext|>


In [ ]:
text = "Can you for one hot minute stop being so based"

In [ ]:
from transformers import GenerationConfig

In [ ]:
generation_config, unused_kwargs = GenerationConfig.from_pretrained(
    "distilgpt2", top_p=50, foo=False, return_unused_kwargs=True
)
generation_config.top_p

50

In [ ]:
input_ids = tokenizer(text, return_tensors="pt")["input_ids"].to(device)
output = model.generate(input_ids, max_new_tokens=256, do_sample=True, generation_config= generation_config)
print(tokenizer.decode(output[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Can you for one hot minute stop being so based on how fast you'll be using it, that the next change is being made. I just gave up on writing for the first time. Sorry. There are a lot of changes that have been made for this, but I would suggest that people, as long as you want to know what will be done, do one quick thing: write, and start experimenting with other stuff, not your traditional method of writing.<|endoftext|>


In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer

model = BartForConditionalGeneration.from_pretrained("facebook/bart-large", forced_bos_token_id=0)
tok = BartTokenizer.from_pretrained("facebook/bart-large")
example_english_phrase = "UN Chief Says There Is No <mask> in Syria"
batch = tok(example_english_phrase, return_tensors="pt")
generated_ids = model.generate(batch["input_ids"])
assert tok.batch_decode(generated_ids, skip_special_tokens=True) == [
    "UN Chief Says There Is No Plan to Stop Chemical Weapons in Syria"
]

In [ ]:
!pip install SentencePiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 41.4 MB/s eta 0:00:00


#Supervised learning:

In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50", src_lang="en_XX", tgt_lang="ro_RO")

src_text = " UN Chief Says There Is No Military Solution in Syria"
tgt_text = "Şeful ONU declară că nu există o soluţie militară în Siria"

model_inputs = tokenizer(src_text, text_target=tgt_text, return_tensors="pt")

model(**model_inputs)  # forward pass

Seq2SeqLMOutput(loss=tensor(9.9719, grad_fn=<NllLossBackward0>), logits=tensor([[[ 5.9384e+01, -1.4623e+00,  3.7075e+01,  ...,  5.7301e+00,
          -9.7913e-01,  1.5201e+01],
         [ 5.9363e+01, -1.4596e+00,  3.6913e+01,  ...,  5.7274e+00,
          -9.5642e-01,  1.5132e+01],
         [ 1.0301e+01, -2.2473e-01,  1.5943e+01,  ..., -2.4285e+00,
           1.6040e+00,  7.8643e+00],
         ...,
         [ 5.3965e-02, -2.3913e-01,  1.2515e+01,  ...,  3.2413e-01,
           1.8379e+00,  5.7910e+00],
         [ 2.5746e+00, -1.8596e-01,  9.4285e+00,  ...,  4.8608e-01,
           1.5175e+00,  4.5009e+00],
         [ 1.1468e+01, -4.1692e-01,  2.4934e+01,  ...,  3.0821e-01,
           1.2864e+00,  1.1943e+01]]], grad_fn=<AddBackward0>), past_key_values=None, decoder_hidden_states=None, decoder_attentions=None, cross_attentions=None, encoder_last_hidden_state=tensor([[[ 0.0204,  0.0117, -0.0101,  ..., -0.0427, -0.0093,  0.0264],
         [-0.8181,  0.4772, -0.7508,  ..., -1.2650,  0.7026, -

#Multilanguage translation models:

In [ ]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast

article_hi = "संयुक्त राष्ट्र के प्रमुख का कहना है कि सीरिया में कोई सैन्य समाधान नहीं है"
article_ar = "الأمين العام للأمم المتحدة يقول إنه لا يوجد حل عسكري في سوريا."

model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")

# translate Arabic to English
tokenizer.src_lang = "ar_AR"
encoded_ar = tokenizer(article_ar, return_tensors="pt")
generated_tokens = model.generate(**encoded_ar, forced_bos_token_id=tokenizer.lang_code_to_id["en_XX"])
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
# => "The Secretary-General of the United Nations says there is no military solution in Syria."

['The Secretary-General of the United Nations says there is no military solution in Syria.']

In [ ]:
# translate Hindi to French
tokenizer.src_lang = "hi_IN"
encoded_hi = tokenizer(article_hi, return_tensors="pt")
generated_tokens = model.generate(**encoded_hi, forced_bos_token_id=tokenizer.lang_code_to_id["fr_XX"])
tokenizer.batch_decode(generated_tokens, skip_special_tokens=True)
# => "Le chef de l 'ONU affirme qu 'il n 'y a pas de solution militaire en Syria."

["Le chef de l 'ONU affirme qu 'il n 'y a pas de solution militaire en Syria."]

#Summarisation

In [ ]:
from transformers import AutoTokenizer, BartModel
import torch

tokenizer = AutoTokenizer.from_pretrained("facebook/bart-base")
model = BartModel.from_pretrained("facebook/bart-base")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state

In [ ]:
from transformers import AutoTokenizer, BartForConditionalGeneration

model = BartForConditionalGeneration.from_pretrained("facebook/bart-large-cnn")
tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

ARTICLE_TO_SUMMARIZE = (
    "After weeks of searching, the trio hit what they thought was pay dirt. Their instruments detected intense radioactivity in brownish-red veins of ore exposed in a rocky outcrop within California’s Clark Mountain Range. But instead of uranium, the brownish-red stuff turned out to be bastnaesite, a mineral bearing fluorine, carbon and 17 curious elements known collectively as rare earths. Traces of radioactive thorium, also in the ore, had set the Geiger counters pinging. "
)
inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=1024, return_tensors="pt")

# Generate Summary
summary_ids = model.generate(inputs["input_ids"], num_beams=2, min_length=0, max_length=20)
tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


'The ore is a mineral bearing fluorine, carbon and 17 curious elements known collectively'

In [ ]:
ARTICLE_TO_SUMMARIZE = (
"For the next few decades, the site, later dubbed Mountain Pass mine, was the world top source for rare earth elements" 
"until two pressures became too much. By the late 1980s" 
"China was intensively mining its own rare earths — and selling them at lower prices. "
"And a series of toxic waste spills at Mountain Pass brought production at the struggling mine to a halt in 2002.")
inputs = tokenizer([ARTICLE_TO_SUMMARIZE], max_length=2048, return_tensors="pt")

# Generate Summary
summary_ids = model.generate(inputs["input_ids"], num_beams=4, min_length=0, max_length=60)
tokenizer.batch_decode(summary_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0]

'For the next few decades, the site, later dubbed Mountain Pass mine, was the world top source for rare earth elements. By the late 1980sChina was intensively mining its own rare earths — and selling them at lower prices. And a series of toxic waste spills at Mountain'